# 量子变分自编码器

<em> Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved. </em>

## 概览

在这个案例中，我们将展示如何训练量子自编码器来压缩和重建给定的量子态（混合态）[1]。

### 原理

量子自编码器的形式与经典自编码器非常相似，同样由编码器 $E$（encoder）和解码器 $D$（decoder）组成。对于输入的 $N$ 量子比特系统的量子态 $\rho_{in}$（这里我们采用量子力学的密度算符表示来描述混合态），先用编码器 $E = U(\theta)$ 将信息编码到其中的部分量子比特上。这部分量子比特记为**系统 $A$**。对剩余的量子比特 （这部分记为**系统 $B$**）进行测量并丢弃后，我们就得到了压缩后的量子态 $\rho_{encode}$！压缩后的量子态维度和量子系统 $A$ 的维度大小保持一致。假设我们需要 $N_A$ 个量子比特来描述系统 $A$ ，那么编码后量子态 $\rho_{encode}$ 的维度就是 $2^{N_A}\times 2^{N_A}$。 这里比较特殊的是, 对应我们这一步测量并丢弃的操作的数学操作是 partial trace。读者在直观上可以理解为张量积 $\otimes$ 的逆运算。

我们不妨看一个具体的例子来理解。给定 $N_A$ 个量子比特上的一个量子态 $\rho_A$ 和 $N_B$ 个量子比特上的一个量子态 $\rho_B$, 那么由 $A、B$ 两个子系统构成的的整体量子系统 $N = N_A+ N_B$ 的量子态就可以描述为: $\rho_{AB} = \rho_A \otimes \rho_B$。现在我们让整个量子系统在酉矩阵 $U$ 的作用下演化一段时间后，得到了一个新的量子态 $\tilde{\rho_{AB}} = U\rho_{AB}U^\dagger$。 那么如果这时候我们只想得到量子子系统 A 所处于的新的量子态 $\tilde{\rho_A}$， 我们该怎么做呢？很简单，只需要测量量子子系统 $B$ 然后将其丢弃。运算上这一步由 partial trace 来完成 $\tilde{\rho_A} = \text{Tr}_B (\tilde{\rho_{AB}})$。在量桨上，我们可以用内置的 `partial_trace(rho_AB, 2**N_A, 2**N_B, 2)` 指令来完成这一运算。**注意：** 其中最后一个参数为 2，表示我们想丢弃量子系统 $B$ 的量子态。

![QA-fig-encoder_pipeline](./figures/QA-fig-encoder_pipeline.png "**图 1.** 量子变分自编码器流程图")

在讨论完编码的过程后，我们来看看如何解码。为了解码量子态 $\rho_{encode}$，我们需要引入与系统 $B$ 维度相同的系统 $C$ 并且初始态取为 $|0\dots0\rangle$ 态。再用解码器 $D = U^\dagger(\theta)$ 作用在整个量子系统 $A+C$ 上对系统 A 中压缩的信息进行解码。我们希望最后得到的量子态 $\rho_{out}$ 与 $\rho_{in}$ 尽可能相似并用 Uhlmann-Josza 保真度 $F$ （Fidelity）来衡量他们之间的相似度。

$$
F(\rho_{in}, \rho_{out}) = \left(\operatorname{tr} \sqrt{\sqrt{\rho_{in}} \rho_{out} \sqrt{\rho_{in}}} \right)^{2}.
\tag{1}
$$

最后，通过优化编码器里的参数，我们就可以尽可能地提高 $\rho_{in}$ 与 $\rho_{out}$ 的保真度。

## Paddle Quantum 实现

下面我们通过一个简单的例子来展示量子自编码器的工作流程和原理。这里我们先引入必要的 package。

In [1]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [6]:
import numpy as np
import paddle
from paddle import matmul, trace, kron, real
import paddle_quantum as pq
from paddle_quantum.ansatz.circuit import Circuit
from paddle_quantum.qinfo import state_fidelity, partial_trace
from paddle_quantum.linalg import dagger, haar_unitary
from paddle_quantum.state.common import to_state

### 生成初始态

我们考虑 $N = 3$ 个量子比特上的量子态 $\rho_{in}$。我们先通过编码器将其信息编码到下方的两个量子比特（系统 $A$），之后对第一个量子比特（系统 $B$）测量并丢弃，之后引入一个处于 $|0\rangle$ 态的量子比特（新的参考系统 $C$）来替代丢弃的量子比特 $B$ 的维度。最后通过解码器，将 A 中压缩的信息复原成 $\rho_{out}$。在这里，我们假设初态是一个混合态并且 $\rho_{in}$ 的本征谱为 $\lambda_i \in \{0.4， 0.2， 0.2， 0.1， 0.1, \,0, \,0, \,0 \}$，然后通过作用一个随机的酉变换来生成初态 $\rho_{in}$。

In [7]:
N_A = 2                          # 系统 A 的量子比特数
N_B = 1                          # 系统 B 的量子比特数
N = N_A + N_B                    # 总的量子比特数
SEED = 15                        # 设置随机数种子
paddle.seed(SEED)
pq.set_dtype('complex64')        # 设置数据类型

V = haar_unitary(N).numpy()                              # 随机生成一个酉矩阵
D = np.diag([0.4, 0.2, 0.2, 0.1, 0.1, 0, 0, 0])          # 输入目标态 rho 的谱
V_H = V.conj().T                                         # 进行厄尔米特转置
rho_in = to_state((V @ D @ V_H).astype('complex64'))     # 生成输入量子态 rho_in
rho_C = to_state(np.diag([1,0]).astype('complex64'))     # 生成辅助量子系统 C

### 搭建量子神经网络

在这里，我们用量子神经网络来作为编码器和解码器。假设系统 A 有 $N_A$ 个量子比特，系统 $B$ 和 $C$ 各有$N_B$ 个量子比特，量子神经网络的深度为 $D$。编码器 $E$ 作用在系统 A 和 B 共同构成的总系统上，解码器 $D$ 作用在 $A$ 和 $C$ 共同构成的总系统上。在我们的问题里，$N_{A} = 2$，$N_{B} = 1$。



In [8]:
# 设置电路深度
cir_depth = 6

# 用 Circuit 类搭建编码器 Encoder E
cir_Encoder = Circuit(N)
for _ in range(cir_depth):
    cir_Encoder.ry('full')
    cir_Encoder.rz('full')
    cir_Encoder.cnot('cycle')
print("初始化的电路：") 
print(cir_Encoder)

初始化的电路：
--Ry(2.974)----Rz(3.296)----*---------x----Ry(4.201)----Rz(4.559)----*---------x----Ry(5.254)----Rz(4.834)----*---------x----Ry(3.263)----Rz(1.664)----*---------x----Ry(5.240)----Rz(1.166)----*---------x----Ry(5.038)----Rz(0.564)----*---------x--
                            |         |                              |         |                              |         |                              |         |                              |         |                              |         |  
--Ry(2.407)----Rz(3.514)----x----*----|----Ry(6.279)----Rz(4.675)----x----*----|----Ry(4.986)----Rz(5.080)----x----*----|----Ry(2.845)----Rz(2.662)----x----*----|----Ry(0.015)----Rz(0.052)----x----*----|----Ry(4.341)----Rz(5.329)----x----*----|--
                                 |    |                                   |    |                                   |    |                                   |    |                                   |    |                                   |    |  
--Ry

### 配置训练模型——损失函数

在这里，我们定义的损失函数为 

$$
Loss = 1 - \langle 0...0|\rho_{trash}|0...0\rangle,
\tag{2}
$$

其中 $\rho_{trash}$ 是经过编码后丢弃的 $B$ 系统的量子态。接着我们通过飞桨训练量子神经网络，最小化损失函数。如果损失函数最后达到 0，则输入态和输出态完全相同。这就意味着我们完美地实现了压缩和解压缩，这时初态和末态的保真度为  $F(\rho_{in}, \rho_{out}) = 1$。

In [9]:
# 超参数设置
LR = 0.2       # 设置学习速率
ITR = 100      # 设置迭代次数

class NET(paddle.nn.Layer):
    def __init__(self, cir, rho_in, rho_C, dtype='float32'):
        super(NET, self).__init__()
        # 载入编码器 E
        self.cir = cir
        # 载入输入态 rho_in 和辅助态 rho_C
        self.rho_in = rho_in
        self.rho_C = rho_C
        # 设置可训练参数
        self.theta = cir.parameters()
    
    # 定义损失函数和前向传播机制
    def forward(self):
    
        # 生成编码器 E 和解码器 D 的矩阵形式
        E = self.cir.unitary_matrix()
        E_dagger = dagger(E)
        D = E_dagger
        D_dagger = E

        # 编码量子态 rho_in
        rho_BA = to_state(matmul(matmul(E, self.rho_in.data), E_dagger))
        
        # 取 partial_trace() 获得 rho_encode 与 rho_trash
        rho_encode = to_state(partial_trace(rho_BA, 2 ** N_B, 2 ** N_A, 1))
        rho_trash = to_state(partial_trace(rho_BA, 2 ** N_B, 2 ** N_A, 2))

        # 解码得到量子态 rho_out
        rho_CA = to_state(kron(self.rho_C.data, rho_encode.data))
        rho_out = to_state(matmul(matmul(D, rho_CA.data), D_dagger))
        
        # 通过 rho_trash 计算损失函数
        zero_Hamiltonian = paddle.to_tensor(np.diag([1,0]).astype('complex64'))
        loss = 1 - real(trace(matmul(zero_Hamiltonian, rho_trash.data)))

        return loss, rho_out

# 生成网络
net = NET(cir_Encoder, rho_in, rho_C)
# 一般来说，我们利用 Adam 优化器来获得相对好的收敛
# 当然你可以改成 SGD 或者是 RMS prop
opt = paddle.optimizer.Adam(learning_rate=LR, parameters=net.parameters())

# 优化循环
for itr in range(1, ITR + 1):
    # 前向传播计算损失函数
    loss, rho_out = net()
    # 反向传播极小化损失函数
    loss.backward()
    opt.minimize(loss)
    opt.clear_grad()
    # 计算并打印保真度
    fid = state_fidelity(rho_in, rho_out)
    if itr % 10 == 0:
        print('iter:', itr, 'loss:', '%.4f' % loss, 'fid:', '%.4f' % np.square(fid.item()))
    if itr == ITR:
        print("\n训练后的电路：") 
        print(cir_Encoder)

iter: 10 loss: 0.1594 fid: 0.8286
iter: 20 loss: 0.1214 fid: 0.8741
iter: 30 loss: 0.1078 fid: 0.8861
iter: 40 loss: 0.1026 fid: 0.8906
iter: 50 loss: 0.1017 fid: 0.8905
iter: 60 loss: 0.1009 fid: 0.8925
iter: 70 loss: 0.1005 fid: 0.8935
iter: 80 loss: 0.1004 fid: 0.8935
iter: 90 loss: 0.1002 fid: 0.8942
iter: 100 loss: 0.1001 fid: 0.8949

训练后的电路：
--Ry(3.020)----Rz(4.904)----*---------x----Ry(3.350)----Rz(4.894)----*---------x----Ry(6.877)----Rz(4.309)----*---------x----Ry(3.806)----Rz(1.201)----*---------x----Ry(4.422)----Rz(1.580)----*---------x----Ry(3.051)----Rz(-0.90)----*---------x--
                            |         |                              |         |                              |         |                              |         |                              |         |                              |         |  
--Ry(2.135)----Rz(3.829)----x----*----|----Ry(5.740)----Rz(3.472)----x----*----|----Ry(5.583)----Rz(4.673)----x----*----|----Ry(2.273)----Rz(2.103)----x----

如果系统 A 的维度为 $d_A$，容易证明量子自编码器能实现的压缩重建最大保真度为 $\rho_{in}$ 最大的 $d_A$ 个本征值之和。在我们的示例里 $d_A = 4$，理论最大保真度为 

$$
F_{\text{max}}(\rho_{in}, \rho_{out})  = \sum_{j=1}^{d_A} \lambda_j(\rho_{in})= 0.4 + 0.2 + 0.2 + 0.1 = 0.9.
\tag{3}
$$

通过 100 次迭代，我们训练出的量子自编码器保真度达到 0.89 以上，和理论最大值非常接近。



_______

## 参考文献

[1] Romero, J., Olson, J. P. & Aspuru-Guzik, A. Quantum autoencoders for efficient compression of quantum data. [Quantum Sci. Technol. 2, 045001 (2017).](https://iopscience.iop.org/article/10.1088/2058-9565/aa8072)

